# test-workbench

Testing deployment of RStudio IDE on Vertex AI Workbench

## Setup

### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [5]:
import os
# Get your Google Cloud project ID from gcloud
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null

try:
    PROJECT_ID = shell_output[0]
except IndexError:
    PROJECT_ID = None

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  demos-vertex-ai


### Enable APIS 

First, you must enable the following APIs for your project:

* Artifact Registry API 
* [Cloud Build API](https://console.cloud.google.com/flows/enableapi?apiid=cloudbuild.googleapis.com)

Learn more about [Enabling service](https://cloud.google.com/artifact-registry/docs/enable-service)

In [ ]:
! gcloud services enable artifactregistry.googleapis.com
! gcloud services enable cloudbuild.googleapis.com

## Build and push container image

### Create a private Docker repository

Your first step is to create your own Docker repository in Google Artifact Registry.

1.  Run the `gcloud artifacts repositories create` command to create a new Docker repository with your region with the description "docker repository".
    
2.  Run the `gcloud artifacts repositories list` command to verify that your repository was created.

In [1]:
REGION = "us-central1" 
PRIVATE_REPO = "rstudio"

In [ ]:
! gcloud artifacts repositories create $PRIVATE_REPO --repository-format=docker \
    --location=$REGION \
    --description="Docker repository for RStudio on Vertex Workbench"

In [2]:
! gcloud artifacts repositories list

Listing items under project demos-vertex-ai, across all locations.

                                                                                     ARTIFACT_REGISTRY
REPOSITORY      FORMAT  MODE                 DESCRIPTION                                        LOCATION     LABELS  ENCRYPTION          CREATE_TIME          UPDATE_TIME          SIZE (MB)
asia.gcr.io     DOCKER  STANDARD_REPOSITORY                                                     asia                 Google-managed key  2022-10-18T19:20:32  2022-10-18T19:20:32  0
automl-beans    DOCKER  STANDARD_REPOSITORY  Docker repository                                  us-central1          Google-managed key  2022-10-19T20:07:45  2022-10-28T19:53:13  2939.842
eu.gcr.io       DOCKER  STANDARD_REPOSITORY                                                     europe               Google-managed key  2022-10-18T19:20:25  2022-10-18T19:20:25  0
gcr.io          DOCKER  STANDARD_REPOSITORY                                               

In [3]:
! gcloud auth configure-docker $REGION-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
Docker configuration file updated.


### Build the Docker container

Next, you build the Docker container image on Cloud Build -- the serverless CI/CD platform.

_Note:_ Building the Docker container image will take ~15 minutes.

In [6]:
IMAGE_NAME = "rstudio"
IMAGE_TAG = "latest" 
IMAGE_URI = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/{IMAGE_NAME}:{IMAGE_TAG}"
IMAGE_URI

'us-central1-docker.pkg.dev/demos-vertex-ai/rstudio/rstudio:latest'

In [ ]:
! gcloud builds submit ./Docker --region=$REGION --tag=$IMAGE_URI --timeout=1h

## Create Vertex AI Workbench Notebook

Start a new Notebook instance from the previously created container image. Make sure to choose a [custom container](https://cloud.google.com/ai-platform/notebooks/docs/custom-container). Pass in the gcr.io address you created earlier. Here's the url:

In [8]:
REPO_URI = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/{IMAGE_NAME}"
REPO_URI

'us-central1-docker.pkg.dev/demos-vertex-ai/rstudio/rstudio'

There are 2 ways to create a notebook instance 

1. Console UI 
2. `gcloud` command (below)

_Note:_ Creating the Workbench Notebook instance the first time will take ~5 minutes due to installation of dependencies

In [ ]:
# TODO - cleanup to be correct after testing complete
# ! gcloud notebooks instances create rstudio-instance --container-repository=$REPO_URI --container-tag=$IMAGE_TAG --machine-type=n1-standard-4 --location=us-central1-b

## Troubleshooting

Troubleshooting to get notebook working

### Download conatiner and run locally 

Download container and run locally to help debug

In [ ]:
# notebook instance/here 
! gcloud auth configure-docker $REGION-docker.pkg.dev --quiet

In [7]:
! docker pull \
    us-central1-docker.pkg.dev/demos-vertex-ai/rstudio/rstudio:latest

latest: Pulling from demos-vertex-ai/rstudio/rstudio

d16dc43b: Pulling fs layer 
ccc3b36b: Pulling fs layer 
be3a151c: Pulling fs layer 
b700ef54: Pulling fs layer 
6d6dc2fb: Pulling fs layer 
ba7a26ce: Pulling fs layer 
1152ebb5: Pulling fs layer 
6ca4c6fa: Pulling fs layer 
189a10f5: Pulling fs layer 
619a13bc: Pulling fs layer 
2bd4fd2a: Pulling fs layer 
729e5c24: Pulling fs layer 
7763aadd: Pulling fs layer 
b689cc10: Pulling fs layer 
50a35261: Pulling fs layer 
7914511d: Pulling fs layer 
b1e50e24: Pulling fs layer 
985fe809: Pulling fs layer 
833c5b8a: Pulling fs layer 
c6c02ab1: Pulling fs layer 
b31e18f2: Pulling fs layer 
5eb60a51: Pulling fs layer 
207f9a7c: Pulling fs layer 
692cef5b: Pulling fs layer 
c2aee86b: Pulling fs layer 
a507301c: Pulling fs layer 
0c71ce80: Pulling fs layer 
e9189cf0: Pulling fs layer 
03e10cc9: Pulling fs layer 
c50aa9f7: Pulling fs layer 
25d267e8: Pulling fs layer 
028736ad: Pulling fs layer 
508b2381: Pulling fs layer 
d527beb4: Pulling fs l

In [ ]:
# cloud shell
# gcloud auth configure-docker us-central1-docker.pkg.dev --quiet
# docker pull us-central1-docker.pkg.dev/demos-vertex-ai/rstudio/rstudio:latest